In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cpu')

In [5]:
input_size = 784
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [11]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, 
                                  transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, 
                                  transform=torchvision.transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size , shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size , shuffle=False)

examples = iter(train_loader)
samples , label = next(examples)
print(samples.shape, label.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [13]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()   
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)

In [14]:
model

NeuralNet(
  (l1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=100, out_features=10, bias=True)
)

In [18]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [24]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}, loss = {loss.item()}')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28*28)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            n_samples +=labels.shape[0]
            n_correct +=(predictions == labels).sum().item()
        acc = 100.0 * (n_correct / n_samples)
        print(f'accuracy = {acc}')
        

epoch 1 / 2, step 100, loss = 0.012173068709671497
epoch 1 / 2, step 200, loss = 0.0032773958519101143
epoch 1 / 2, step 300, loss = 0.003243264276534319
epoch 1 / 2, step 400, loss = 0.08177801966667175
epoch 1 / 2, step 500, loss = 0.008354736492037773
epoch 1 / 2, step 600, loss = 0.001699834712781012
accuracy = 97.7
epoch 2 / 2, step 100, loss = 0.003287692554295063
epoch 2 / 2, step 200, loss = 0.0121592553332448
epoch 2 / 2, step 300, loss = 0.004739112686365843
epoch 2 / 2, step 400, loss = 0.002801761729642749
epoch 2 / 2, step 500, loss = 0.0018375213257968426
epoch 2 / 2, step 600, loss = 0.011126740835607052
accuracy = 97.74000000000001
